In [30]:
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils import data  # 获取迭代数据
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torch.autograd import Variable  # 获取变量

import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchensemble import VotingClassifier, FusionClassifier

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import datetime
import time

In [17]:
#setting
batch_size = 16
learning_rate = 0.01
epochs = 10
clip = 0.01

no_cuda = False
use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

## load data

In [18]:
all_data = np.load("../input_data/data.npy")
all_label = np.load("../input_data/label.npy")

In [19]:
all_data = torch.from_numpy(all_data).to(device)
all_label = torch.from_numpy(all_label).to(device)
all_data = all_data.float()
all_label = all_label.long()
#pack to dataset
dataset = TensorDataset(all_data, all_label)


In [20]:
#划分训练集和测试集

seed = 30
num_train_all = int(len(dataset) * 0.9)
num_test = len(dataset) - num_train_all
num_train = int(num_train_all * 0.8)
num_val = num_train_all - num_train

train_dataset, validate_dataset, test_dataset = random_split(dataset, [num_train, num_val, num_test],
                                                             torch.Generator().manual_seed(seed))

In [21]:
#Load to DataLoader
print("train_dataset:", len(train_dataset))
print("validate_dataset:", len(validate_dataset))
print("test_dataset:", len(test_dataset))
print("batch_size:", batch_size)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset=validate_dataset, batch_size=batch_size, shuffle=True)


train_dataset: 11875
validate_dataset: 2969
test_dataset: 1650
batch_size: 16


## Model

In [22]:
class EagleC_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        #卷积
        self.features_ = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
                                       , nn.ReLU(inplace=True)
                                       , nn.MaxPool2d(2)

                                       , nn.Conv2d(32, 64, 3, stride=1, padding=1)
                                       , nn.ReLU(inplace=True)
                                       , nn.MaxPool2d(2)
                                       )
        #分类
        #根据net输出的形状确定
        self.clf_ = nn.Sequential(nn.Dropout(0.5)
                                  , nn.Linear(64 * 5 * 5, 512)
                                  , nn.ReLU(inplace=True)
                                  , nn.Linear(512, 6)
                                  , nn.Sigmoid()
                                  )

    def forward(self, x):
        x = self.features_(x)  #用特征提取的架构提取特征
        x = x.view(-1, 64 * 5 * 5)  #调整数据结构，拉平数据
        output = self.clf_(x)
        return output

In [26]:
model = VotingClassifier(
    estimator=EagleC_CNN,
    n_estimators=5,
    cuda=False,
).to(device)

# model = EagleC_CNN().to(device)

In [27]:

#多分类
criterion = nn.CrossEntropyLoss()
model.set_criterion(criterion)
#优化器
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.set_optimizer('Adam',  # parameter optimizer
                    lr=1e-3,  # learning rate of the optimizer
                    weight_decay=5e-4)  # weight decay of the optimizer

## Train

In [28]:
# scheduler = ReduceLROnPlateau(optimizer, 'min')

In [31]:
time0 = time.time()
model.fit(train_loader=train_loader,  # training data
          epochs=4)
print("Use {} s".format(time.time() - time0))

Estimator: 000 | Epoch: 000 | Batch: 000 | Loss: 1.79369 | Correct: 2/16
Estimator: 000 | Epoch: 000 | Batch: 100 | Loss: 1.61582 | Correct: 6/16
Estimator: 000 | Epoch: 000 | Batch: 200 | Loss: 1.52333 | Correct: 5/16
Estimator: 000 | Epoch: 000 | Batch: 300 | Loss: 1.55128 | Correct: 4/16
Estimator: 000 | Epoch: 000 | Batch: 400 | Loss: 1.47394 | Correct: 6/16
Estimator: 000 | Epoch: 000 | Batch: 500 | Loss: 1.53665 | Correct: 7/16
Estimator: 000 | Epoch: 000 | Batch: 600 | Loss: 1.48053 | Correct: 9/16
Estimator: 000 | Epoch: 000 | Batch: 700 | Loss: 1.73456 | Correct: 3/16
Estimator: 001 | Epoch: 000 | Batch: 000 | Loss: 1.78374 | Correct: 9/16
Estimator: 001 | Epoch: 000 | Batch: 100 | Loss: 1.59709 | Correct: 5/16
Estimator: 001 | Epoch: 000 | Batch: 200 | Loss: 1.40919 | Correct: 6/16
Estimator: 001 | Epoch: 000 | Batch: 300 | Loss: 1.52196 | Correct: 6/16
Estimator: 001 | Epoch: 000 | Batch: 400 | Loss: 1.45946 | Correct: 5/16
Estimator: 001 | Epoch: 000 | Batch: 500 | Loss: 1.

In [46]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [47]:
accuracy = model.evaluate(test_loader)

ValueError: The type of input X should be one of {{torch.Tensor, np.ndarray}}.